In [30]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [43]:
all_label_folders = [dir for dir in os.listdir('labels') if 'Folder' in dir]
print('reading...')
for label_folder in all_label_folders:
    video_path = f"training_videos/{label_folder[7:]}.avi"
    if os.path.isfile(video_path):
        print(video_path)
        cap = cv2.VideoCapture(video_path)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frames = []
        for i in range(length):
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
        frames = np.array(frames)

reading...
training_videos/8 W.avi
training_videos/20 W.avi
training_videos/21 W.avi
training_videos/10 W.avi
training_videos/9 W.avi
training_videos/7 W.avi


In [32]:
class VideoDataset:
    def __init__(self, label_dir, video_dir, sep) -> None:
        self.all_label_folders = [dir for dir in os.listdir(label_dir) if 'Folder' in dir]
        self.label_dir = label_dir
        self.video_dir = video_dir
        self.sep = sep
    
    def __len__(self):
        return len(self.all_label_folders)
    
    def __getitem__(self, idx):
        folder_name = self.all_label_folders[idx]
        video_path = f"training_videos/{label_folder[7:]}.avi"
        if not os.path.isfile(video_path):
            return None
        cap = cv2.VideoCapture(video_path)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frames = []
        for i in range(length):
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
        frames = np.array(frames)
        anno_path = os.path.join(self.label_dir, folder_name)
        if not os.path.isdir(anno_path):
            return None
        anno_path = [path for path in os.listdir(anno_path) if (not '.zip' in path) and ('YOLO' in path)]
        assert len(anno_path) == 0
        anno_path = anno_path[0]
        annos = []
        for file in os.listdir(anno_path):
            if not '.txt' in file:
                continue
            anno = {'labels':[], 'boxes':[]}
            with open(file) as f:
                lines = f.readlines()
                for line in lines:
                    ll = line.split(' ')
                    anno['labels'].append(ll[0])
                    anno['boxes'].append(ll[1:])
            annos.append(anno)
        all_annos = []
        for i in range(length):
            id = i/self.sep
            all_annos.append(annos[id])
        return frames, all_annos